# 01 Demo Environment | 01 Create Azure Environment

This notebook contains the script to create the necessary Azure environment to run the provided samples. The notebook uses [PowerShell](https://learn.microsoft.com/powershell/scripting/install/installing-powershell?view=powershell-7.4) and [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) to deploy all necessary Azure resources. Both tools are available on Windows, macOS and Linux environments.

This notebook performs two tasks:

- Deployment of necessary Azure Services (Azure OpenAI, Azure AI Search ...) to run the provided samples
- Store all necessary service endpoints, service API keys, Azure OpenAI deployment names in a **centralized env file** (./Configuration/application.env). This file is used by all notebooks in this repo to connect and authenticate against the deployed Azure services.

If you already have instances of Azure OpenAI, Azure AI Search running you can rename the [configuration template](../conf/application.env.example) to `application.env` and provide endpoint, API key and deployment names of a chat completion and an embedding model and the other needed services. We suggest to run the notebook to start a clean environment.

## Step 1:   Login to Azure; Get, Set subscription

In [1]:
# Check if you are already logged in
$loggedIn = az account show --query "name" -o tsv

if ($loggedIn -ne $null) {
    Write-Host "Already logged in as $loggedIn"
} else {
    Write-Host "Logging in..."
    az login
}
# Retrieve default subscription id
$subscriptionId = (
    (
        az account list -o json `
            --query "[?isDefault]"
    ) | ConvertFrom-Json
).id

# Set Subscription
az account set --subscription $subscriptionId
Write-Host "Subscription set to $subscriptionId"

Already logged in as RobEichBamiDec2024Subscription
Subscription set to d1ea45fe-b62b-4fb8-9b3b-f718c55a3330


## Step 2:   Define project unifier

The project unifier is used to allow multiple deployments of services which have a need for a unique custom endpoint.

In [2]:
# Define Project Unifier

$random = Get-Random -Minimum 100 -Maximum 999

Write-Host "Unifier set to: $random"

Unifier set to: 214


## Step 3:   Create Resource Group

In this sample all resources are deployed to `swedencentral`. Feel free to change to your preferred location.

In [3]:
# Create Resource Group
$resourceGroup = "OpenAI.Workshop"
$location = "swedencentral"

az group create `
    --location $location `
    --resource-group $resourceGroup

{
  "id": "/subscriptions/d1ea45fe-b62b-4fb8-9b3b-f718c55a3330/resourceGroups/OpenAI.Workshop",
  "location": "swedencentral",
  "managedBy": null,
  "name": "OpenAI.Workshop",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


## Step 4:   Create Azure OpenAI instance

An instance of Azure Cognitive Service with the kind `OpenAI` will be created. The `endpoint` and `API key` of the newly created instance are retrieved for later storage in the `application.env` file.

In [4]:
# Create Azure OpenAI Account
$csOpenAIName = "WS_AOAI_$random"

az cognitiveservices account create `
    --name $csOpenAIName `
    --resource-group $resourceGroup `
    --location $location `
    --kind OpenAI `
    --sku S0 `
    --yes

$csOpenAIId = ( `
    az cognitiveservices account show `
        --name $csOpenAIName `
        --resource-group $resourceGroup `
        --query id `
        --output tsv `
)

$csOpenAIEndpoint = ( `
    az cognitiveservices account show `
        --name $csOpenAIName `
        --resource-group $resourceGroup `
        --query properties.endpoint `
        --output tsv `
)

$csOpenAIApiKey = (
    az cognitiveservices account keys list `
        --name $csOpenAIName `
        --resource-group $resourceGroup `
        --query key1 `
        --output tsv `
)


{
  "etag": "\"1301c009-0000-4700-0000-67bd739b0000\"",
  "id": "/subscriptions/d1ea45fe-b62b-4fb8-9b3b-f718c55a3330/resourceGroups/OpenAI.Workshop/providers/Microsoft.CognitiveServices/accounts/WS_AOAI_214",
  "identity": null,
  "kind": "OpenAI",
  "location": "swedencentral",
  "name": "WS_AOAI_214",
  "properties": {
    "abusePenalty": null,
    "allowedFqdnList": null,
    "apiProperties": {
      "aadClientId": null,
      "aadTenantId": null,
      "additionalProperties": null,
      "eventHubConnectionString": null,
      "qnaAzureSearchEndpointId": null,
      "qnaAzureSearchEndpointKey": null,
      "qnaRuntimeEndpoint": null,
      "statisticsEnabled": null,
      "storageAccountConnectionString": null,
      "superUser": null,
      "websiteName": null
    },
    "callRateLimit": {
      "count": null,
      "renewalPeriod": null,
      "rules": [
        {
          "count": 30.0,
          "dynamicThrottlingEnabled": null,
          "key": "openai.dalle.post",
          

If you get an error message saying **"... you have to accept terms for responsible AI ..."** then most probably this is the first time you deploy an Azure OpenAI instance. In this case please use the [Azure Portal](https://portal.azure.com) to provision the Azure OpenAI instance and accept the terms for responsible AI.

## Step 5:   Deploy Azure OpenAI models

Two LLM models are deployed to the newly created Azure Cognitive Service instance: 

- A chat completion model. In the sample we're deploying `gpt-4o`. This can be replaced with other models providing a chat completion interface like `gpt-35-turbo` or others.
- A text embedding model. In the sample we're deploying `text-embedding-ada-002`. Any other text embedding model can be deployed as well.

In [5]:
# Chat Completion Model
$modelChatCompletionDeploymentName = "gpt-4o"
$modelName = "gpt-4o"
$modelVersion = "2024-05-13"
$modelFormat = "OpenAI"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $modelChatCompletionDeploymentName `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1


{
  "etag": "\"4b5f2e0f-1fdb-4dda-a6ec-4f11e3b23450\"",
  "id": "/subscriptions/d1ea45fe-b62b-4fb8-9b3b-f718c55a3330/resourceGroups/OpenAI.Workshop/providers/Microsoft.CognitiveServices/accounts/WS_AOAI_214/deployments/gpt-4o",
  "name": "gpt-4o",
  "properties": {
    "callRateLimit": null,
    "capabilities": {
      "area": "EUR",
      "assistants": "true",
      "chatCompletion": "true",
      "jsonObjectResponse": "true",
      "maxContextToken": "128000",
      "maxOutputToken": "4096"
    },
    "model": {
      "callRateLimit": null,
      "format": "OpenAI",
      "name": "gpt-4o",
      "source": null,
      "version": "2024-05-13"
    },
    "provisioningState": "Succeeded",
    "raiPolicyName": null,
    "rateLimits": [
      {
        "count": 1.0,
        "dynamicThrottlingEnabled": null,
        "key": "request",
        "matchPatterns": null,
        "minCount": null,
        "renewalPeriod": 10.0
      },
      {
        "count": 1000.0,
        "dynamicThrottlingEnab

In [6]:
# Text Embedding Model
$modelFormat = "OpenAI"
$modelEmbeddingDeploymentName = "textembedding-ada-002"
$modelName = "text-embedding-ada-002"
$modelVersion = "2"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $modelEmbeddingDeploymentName `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1



{
  "etag": "\"a72f0eaa-d6d0-481f-8ad9-c31a8474a07c\"",
  "id": "/subscriptions/d1ea45fe-b62b-4fb8-9b3b-f718c55a3330/resourceGroups/OpenAI.Workshop/providers/Microsoft.CognitiveServices/accounts/WS_AOAI_214/deployments/textembedding-ada-002",
  "name": "textembedding-ada-002",
  "properties": {
    "callRateLimit": null,
    "capabilities": {
      "embeddings": "true",
      "embeddingsMaxInputs": "2048"
    },
    "model": {
      "callRateLimit": null,
      "format": "OpenAI",
      "name": "text-embedding-ada-002",
      "source": null,
      "version": "2"
    },
    "provisioningState": "Succeeded",
    "raiPolicyName": null,
    "rateLimits": [
      {
        "count": 1.0,
        "dynamicThrottlingEnabled": null,
        "key": "request",
        "matchPatterns": null,
        "minCount": null,
        "renewalPeriod": 10.0
      },
      {
        "count": 1000.0,
        "dynamicThrottlingEnabled": null,
        "key": "token",
        "matchPatterns": null,
        "minCou

In [7]:
# Dall-E Model
$modelFormat = "OpenAI"
$modelImageDeploymentName = "dall-e-3"
$modelName = "dall-e-3"
$modelVersion = "3.0"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $modelImageDeploymentName `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1


ERROR: (InsufficientQuota) This operation require 1 new capacity in quota Capacity Unit - Dalle, which is bigger than the current available capacity 0. The current quota usage is 1 and the quota limit is 1 for quota Capacity Unit - Dalle.
Code: InsufficientQuota
Message: This operation require 1 new capacity in quota Capacity Unit - Dalle, which is bigger than the current available capacity 0. The current quota usage is 1 and the quota limit is 1 for quota Capacity Unit - Dalle.


## Step 6: Create Azure AI Vision

Azure AI Vision is deployed to use its multi modal embedding capabilities. Specifically the functionality to embed images into a multi vector space and query the image embeddings using text. 

In [8]:
# AI Vision
$csVisionName = "wsvision$random"
$csVisionSku = "S1"

az cognitiveservices account create `
    --name $csVisionName `
    --resource-group $resourceGroup `
    --location $location `
    --kind ComputerVision `
    --sku $csVisionSku `
    --yes

$csVisionEndpoint = ( `
    az cognitiveservices account show `
        --name $csVisionName `
        --resource-group $resourceGroup `
        --query properties.endpoint `
        --output tsv `
)

$csVisionApiKey = ( `
    az cognitiveservices account keys list `
        --name $csVisionName `
        --resource-group $resourceGroup `
        --query key1 `
        --output tsv `
)


{
  "etag": "\"13018611-0000-4700-0000-67bd73e90000\"",
  "id": "/subscriptions/d1ea45fe-b62b-4fb8-9b3b-f718c55a3330/resourceGroups/OpenAI.Workshop/providers/Microsoft.CognitiveServices/accounts/wsvision214",
  "identity": null,
  "kind": "ComputerVision",
  "location": "swedencentral",
  "name": "wsvision214",
  "properties": {
    "abusePenalty": null,
    "allowedFqdnList": null,
    "apiProperties": null,
    "callRateLimit": {
      "count": null,
      "renewalPeriod": null,
      "rules": [
        {
          "count": 30.0,
          "dynamicThrottlingEnabled": true,
          "key": "vision.recognizeText",
          "matchPatterns": [
            {
              "method": "POST",
              "path": "vision/recognizeText"
            },
            {
              "method": "GET",
              "path": "vision/textOperations/*"
            },
            {
              "method": "*",
              "path": "vision/read/*"
            }
          ],
          "minCount": null

## Step 7:   Create Azure AI Search

Azure AI Search is deployed to use its [vector DB functionalities](https://learn.microsoft.com/en-us/azure/search/vector-search-overview). 
Just like with Azure OpenAI Cognitive Service, the `endpoint` and `API key` of the newly created instance are retrieved for later storage in the `application.env` file. The SKU 'free' is used for the instance. Ensure that you haven't yet deployed a free SKU or change to 'standard'

In [9]:
# Azure AI Search
$csSearchName = "ws-aisearch-$random"
$csSearchSku = "free"

az search service create `
    --name $csSearchName `
    --resource-group $resourceGroup `
    --location $location `
    --sku $csSearchSku

$csSearchEndpoint = "https://$csSearchName.search.windows.net"

$csSearchApiKey = ( `
    az search admin-key show `
        --resource-group $resourceGroup `
        --service-name $csSearchName `
        --query primaryKey `
        --output tsv `
)

{
  "authOptions": {
    "apiKeyOnly": {}
  },
  "disableLocalAuth": false,
  "encryptionWithCmk": {
    "encryptionComplianceStatus": "Compliant",
    "enforcement": "Unspecified"
  },
  "hostingMode": "default",
  "id": "/subscriptions/d1ea45fe-b62b-4fb8-9b3b-f718c55a3330/resourceGroups/OpenAI.Workshop/providers/Microsoft.Search/searchServices/ws-aisearch-214",
  "location": "Sweden Central",
  "name": "ws-aisearch-214",
  "networkRuleSet": {
    "ipRules": []
  },
  "partitionCount": 1,
  "privateEndpointConnections": [],
  "provisioningState": "succeeded",
  "publicNetworkAccess": "Enabled",
  "replicaCount": 1,
  "resourceGroup": "OpenAI.Workshop",
  "semanticSearch": "disabled",
  "sharedPrivateLinkResources": [],
  "sku": {
    "name": "free"
  },
  "status": "running",
  "statusDetails": "",
  "type": "Microsoft.Search/searchServices"
}


## Step 8:   Create Azure Storage account

Azure Storage is deployed to store data to e.g. create image embeddings using Azure AI Vision.

In [10]:
# Create Storage Account
$stgName = "wsstorage$random"
$stgSku = "Standard_LRS"

az storage account create `
    --name $stgName `
    --resource-group $resourceGroup `
    --location $location `
    --sku $stgSku `
    --kind StorageV2 `
    --https-only true `
    --access-tier Hot

$stgConnectionString = ( `
    az storage account show-connection-string `
        --name $stgName `
        --resource-group $resourceGroup `
        --query connectionString `
        --output tsv `
)

{
  "accessTier": "Hot",
  "accountMigrationInProgress": null,
  "allowBlobPublicAccess": false,
  "allowCrossTenantReplication": false,
  "allowSharedKeyAccess": null,
  "allowedCopyScope": null,
  "azureFilesIdentityBasedAuthentication": null,
  "blobRestoreStatus": null,
  "creationTime": "2025-02-25T07:41:23.177225+00:00",
  "customDomain": null,
  "defaultToOAuthAuthentication": null,
  "dnsEndpointType": null,
  "enableExtendedGroups": null,
  "enableHttpsTrafficOnly": true,
  "enableNfsV3": null,
  "encryption": {
    "encryptionIdentity": null,
    "keySource": "Microsoft.Storage",
    "keyVaultProperties": null,
    "requireInfrastructureEncryption": null,
    "services": {
      "blob": {
        "enabled": true,
        "keyType": "Account",
        "lastEnabledTime": "2025-02-25T07:41:24.020992+00:00"
      },
      "file": {
        "enabled": true,
        "keyType": "Account",
        "lastEnabledTime": "2025-02-25T07:41:24.020992+00:00"
      },
      "queue": null,
   

## Step 9: Create application.env file

In [11]:
# Store configuration 
$configurationFile = "../Configuration/application.env"
New-Item -Name $configurationFile -ItemType File -Force


function Set-ConfigurationFileVariable($configurationFile, $variableName, $variableValue) {
    if (Select-String -Path $configurationFile -Pattern $variableName) {
        (Get-Content $configurationFile) | Foreach-Object {
            $_ -replace "$variableName = .*", "$variableName = $variableValue"
        } | Set-Content $configurationFile
    } else {
        Add-Content -Path $configurationFile -value "$variableName = $variableValue"
    }
}

Set-ConfigurationFileVariable $configurationFile "WS_AOAI_APIKEY" $csOpenAIApiKey
Set-ConfigurationFileVariable $configurationFile "WS_AOAI_ENDPOINT" $csOpenAIEndpoint
Set-ConfigurationFileVariable $configurationFile "WS_CHATCOMPLETION_DEPLOYMENTNAME" $modelChatCompletionDeploymentName
Set-ConfigurationFileVariable $configurationFile "WS_EMBEDDING_DEPLOYMENTNAME" $modelEmbeddingDeploymentName
Set-ConfigurationFileVariable $configurationFile "WS_IMAGE_DEPLOYMENTNAME" $modelImageDeploymentName
Set-ConfigurationFileVariable $configurationFile "WS_VISION_APIKEY" $csVisionApiKey
Set-ConfigurationFileVariable $configurationFile "WS_VISION_ENDPOINT" $csVisionEndpoint
Set-ConfigurationFileVariable $configurationFile "WS_STORAGE_CONNECTIONSTRING" $stgConnectionString
Set-ConfigurationFileVariable $configurationFile "WS_SEARCH_ENDPOINT" $csSearchEndpoint
Set-ConfigurationFileVariable $configurationFile "WS_SEARCH_APIKEY" $csSearchApiKey
Set-ConfigurationFileVariable $configurationFile "WS_ASSETS_FOLDER" "../../assets"


Write-Host "Configuration file created at: $configurationFile"



    Directory: 
C:\Sourcen\GitHubProjects\OpenAI.OneDayWorkshop\01_CreateEnvironment\..\Configuration

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
-a---           2/25/2025  8:42 AM              0 application.env
Configuration file created at: ../Configuration/application.env

